# How to match? 

## 0. Import libraries 

In [5]:
# !pip install faiss-cpu
# !pip install lightautoml

In [6]:
!python --version

Python 3.9.7


In [1]:
import pandas as pd
import warnings
import numpy as np
from lightautoml.addons.hypex import Matcher

warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [2]:
# Simulating dataset with known effect size
num_users = 10000
num_months = 12

signup_months = np.random.choice(np.arange(1, num_months), num_users) * np.random.randint(0,2, size=num_users) # signup_months == 0 means customer did not sign up
df = pd.DataFrame({
    'user_id': np.repeat(np.arange(num_users), num_months),
    'signup_month': np.repeat(signup_months, num_months), # signup month == 0 means customer did not sign up
    'month': np.tile(np.arange(1, num_months+1), num_users), # months are from 1 to 12
    'spend': np.random.poisson(500, num_users*num_months) #np.random.beta(a=2, b=5, size=num_users * num_months)*1000 # centered at 500
})
# A customer is in the treatment group if and only if they signed up
df["treat"] = df["signup_month"]>0
# Simulating an effect of month (monotonically decreasing--customers buy less later in the year)
df["spend"] = df["spend"] - df["month"]*10
# Simulating a simple treatment effect of 100
after_signup = (df["signup_month"] < df["month"]) & (df["treat"])
df.loc[after_signup,"spend"] = df[after_signup]["spend"] + 100
df.head()

,user_id,signup_month,month,spend,treat
0,0,10,1,525,True
1,0,10,2,466,True
2,0,10,3,484,True
3,0,10,4,513,True
4,0,10,5,430,True


In [3]:
# Setting the signup month (for ease of analysis)
i = 3
df_i_signupmonth = (
    df[df.signup_month.isin([0, i])]
    .groupby(["user_id", "signup_month", "treat"])
    .apply(
        lambda x: pd.Series(
            {
                "pre_spends": x.loc[x.month < i, "spend"].mean(),
                "post_spends": x.loc[x.month > i, "spend"].mean(),
            }
        )
    )
    .reset_index()
)
df_i_signupmonth

,user_id,signup_month,treat,pre_spends,post_spends
0,2,0,False,512.0,415.222222
1,3,0,False,503.5,414.777778
2,4,0,False,476.5,416.888889
3,5,0,False,497.5,417.555556
4,8,0,False,487.0,423.333333
...,...,...,...,...,...
5429,9992,0,False,482.5,424.444444
5430,9995,0,False,517.5,419.888889
5431,9996,3,True,504.5,516.555556
5432,9998,0,False,479.0,433.777778


In [4]:
# Additional category features
gender = np.random.choice(a=[0,1], size=df_i_signupmonth.user_id.nunique())
age = np.random.choice(a=range(18, 70), size=df_i_signupmonth.user_id.nunique())
industry = np.random.choice(a=range(1, 3), size=df_i_signupmonth.user_id.nunique())
df_i_signupmonth['age'] = age
df_i_signupmonth['is_male'] =  gender
df_i_signupmonth['industry'] =  industry
df_i_signupmonth['industry'] = df_i_signupmonth['industry'].astype('str')
df_i_signupmonth['treat'] = df_i_signupmonth['treat'].astype(int)
df_i_signupmonth.head()

,user_id,signup_month,treat,pre_spends,post_spends,age,is_male,industry
0,2,0,0,512.0,415.222222,28,1,1
1,3,0,0,503.5,414.777778,42,0,1
2,4,0,0,476.5,416.888889,23,1,2
3,5,0,0,497.5,417.555556,38,0,1
4,8,0,0,487.0,423.333333,32,1,2


In [5]:
df_i_signupmonth.columns

Index(['user_id', 'signup_month', 'treat', 'pre_spends', 'post_spends', 'age',
       'is_male', 'industry'],
      dtype='object')

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [6]:
info_col = ['user_id', 'signup_month']

outcome = 'post_spends'
treatment = 'treat'

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [7]:
# Standard model with base parameters
model = Matcher(
    input_data=df_i_signupmonth, 
    outcome=outcome, 
    treatment=treatment, 
    info_col=info_col, 
    n_neighbors=10, 
    silent=True, 
    pbar=True
)
results, quality_results = model.estimate()

  0%|          | 0/5434 [00:00<?, ?it/s]

In [8]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper
ATE,99.544385,0.604560,0.0,98.359448,100.729321
ATC,99.495295,0.615834,0.0,98.288260,100.702330
ATT,100.101548,0.726892,0.0,98.676840,101.526257


### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [9]:
# group_col is used for strict comparison of categorical features.  
# In our case there is only one attribute  
# If there are several such attributes, you should make one of them and use it

group_col = "industry"

In [10]:
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment, 
                info_col=info_col, group_col=group_col)
results, quality_results = model.estimate()

  0%|          | 0/4 [00:00<?, ?it/s]

### 2.3 Matching but you don't know which features are better to use  
This is the easiest way to initialize and calculate metrics on a Matching task  
Use this method when you want to select the most important features and make matching based on them. 

In [11]:
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment, 
                info_col=info_col, group_col=group_col)

In [12]:
features_importance = model.lama_feature_select()
features_importance

,Feature,Importance
0,pre_spends,623818.811035
3,industry,623818.811035
1,age,490037.698975
2,is_male,82132.000977


In [13]:
features = features_importance['Feature'].to_list()

In [14]:
# You can use both variant that you like. So you can just list the features to be matched

#results, quality_results = model.estimate(features=features_importance[:3])
results, quality_results = model.estimate(features=features[:3])

  0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
model.matcher.df_matched

,index,pre_spends,age,is_male,industry,pre_spends_matched,age_matched,is_male_matched,index_matched,industry_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,4994,472.5,65,1,1,473.0,66.0,0.5,"[10, 756]",1,532.333333,421.888889,110.429087,1,0
1,4995,477.0,67,0,1,477.0,67.0,1.0,[916],1,512.444444,429.777778,82.666667,1,0
2,4996,475.5,54,0,1,476.0,54.0,0.0,[1962],1,528.444444,425.333333,103.099624,1,0
3,4997,474.0,56,1,1,474.5,56.0,1.0,[235],1,524.555556,421.333333,103.210735,1,0
4,4998,520.0,38,0,1,519.5,40.0,1.0,[580],1,514.888889,423.888889,91.003748,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4989,4989,454.0,64,0,2,458.0,58.0,1.0,[433],2,421.333333,513.555556,92.188671,0,1
4990,4990,488.0,33,1,2,488.5,33.0,0.0,[265],2,427.666667,525.222222,97.562135,0,1
4991,4991,468.5,39,0,2,468.5,40.0,0.0,[402],2,427.333333,526.000000,98.681031,0,1
4992,4992,517.5,46,1,2,515.5,53.0,1.0,[244],2,419.888889,516.000000,96.185344,0,1


## 3. Results  
### 3.1 ATE, ATT, ATC

In [16]:
# model.matcher.results
results

,effect_size,std_err,p-val,ci_lower,ci_upper
ATE,99.577072,0.584879,0.0,98.430710,100.723434
ATC,99.552488,0.595559,0.0,98.385193,100.719784
ATT,99.856092,0.703849,0.0,98.476548,101.235637


### 3.2 SMD, PSI, KS-test, repeats

In [17]:
# matching quality result - SMD
model.quality_result

{'psi':         column_treated  anomaly_score_treated check_result_treated  \
 0          age_treated                   0.00                   OK   
 1     industry_treated                   0.00                   OK   
 2  post_spends_treated                  16.11                  NOK   
 3   pre_spends_treated                   0.00                   OK   
 
         column_untreated  anomaly_score_untreated check_result_untreated  
 0          age_untreated                     0.02                     OK  
 1     industry_untreated                     0.00                     OK  
 2  post_spends_untreated                     8.25                    NOK  
 3   pre_spends_untreated                     0.00                     OK  ,
 'ks_test':             match_control_to_treat  match_treat_to_control
 age                       1.000000                0.201820
 is_male                   0.020002                0.000066
 pre_spends                1.000000                0.291413,
 's

In [18]:
# matching quality result - PSI
model.quality_result['psi']

,column_treated,anomaly_score_treated,check_result_treated,column_untreated,anomaly_score_untreated,check_result_untreated
0,age_treated,0.00,OK,age_untreated,0.02,OK
1,industry_treated,0.00,OK,industry_untreated,0.00,OK
2,post_spends_treated,16.11,NOK,post_spends_untreated,8.25,NOK
3,pre_spends_treated,0.00,OK,pre_spends_untreated,0.00,OK


In [19]:
# matching quality result - KS-test

model.quality_result['ks_test']

,match_control_to_treat,match_treat_to_control
age,1.000000,0.201820
is_male,0.020002,0.000066
pre_spends,1.000000,0.291413


In [20]:
# matching quality result - repeats
model.quality_result['repeats']

{'match_control_to_treat': 0.94, 'match_treat_to_control': 0.09}

### 3.3 Validation
validate result with placebo treatment  
our new effect size is close to zero (it should be)

In [21]:
model.validate_result()

  0%|          | 0/10 [00:00<?, ?it/s]

{'post_spends': [99.42747575436294, 0.165547334729794]}

In [22]:
model.matcher.df_matched

,index,pre_spends,age,is_male,industry,random_feature,pre_spends_matched,age_matched,is_male_matched,random_feature_matched,index_matched,industry_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,4994,472.5,65,1,1,0.106094,474.5,63.0,1.0,0.139020,[248],1,532.333333,411.444444,120.900300,1,0
1,4995,477.0,67,0,1,0.255287,478.5,68.0,1.0,0.284811,[1570],1,512.444444,415.111111,97.301547,1,0
2,4996,475.5,54,0,1,0.132842,476.0,56.0,0.0,0.229170,[2007],1,528.444444,421.111111,107.289464,1,0
3,4997,474.0,56,1,1,0.253929,476.0,56.0,0.0,0.229170,[2007],1,524.555556,421.111111,103.428294,1,0
4,4998,520.0,38,0,1,-0.793859,520.5,36.0,0.0,-1.071852,[449],1,514.888889,415.111111,99.826539,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4989,4989,454.0,64,0,2,1.709584,458.0,58.0,1.0,2.099635,[433],2,421.333333,513.555556,92.228598,0,1
4990,4990,488.0,33,1,2,1.769668,489.0,34.0,0.0,1.561221,[379],2,427.666667,525.555556,97.865706,0,1
4991,4991,468.5,39,0,2,0.168121,471.5,42.0,1.0,0.277047,[304],2,427.333333,531.666667,104.341141,0,1
4992,4992,517.5,46,1,2,-2.314767,500.0,38.0,0.0,-2.033592,[359],2,419.888889,511.333333,91.523795,0,1
